In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cisco-automation/PE1-Cisco.txt
/kaggle/input/cisco-automation/Cisco IOS XE Switch RTR Security Technical Implementation Guide-MAC-3_Sensitive.xls


In [2]:
!pip install faiss-cpu sentence-transformers scikit-learn matplotlib seaborn transformers peft accelerate pandas openpyxl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 32.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 23.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install xlrd


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 2.6 MB/s eta 0:00:00ta 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import faiss
import numpy as np
import re
import torch
import json
import pandas as pd
import textwrap

from sentence_transformers import SentenceTransformer
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score
)
import matplotlib.pyplot as plt
import seaborn as sns

# If you want LoRA:
try:
    from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
except ImportError:
    print("Note: peft not installed. LoRA steps will be skipped.")

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
E0000 00:00:1740486724.545627      10 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:230


In [5]:
##############################################################################
# 1) CONFIG
##############################################################################
LEARNING_RATE = 3e-5
LORA_RANK = 8
LORA_ALPHA = 32
TARGET_MODULES = ["q_proj", "v_proj"]
BATCH_SIZE = 2
ACCUMULATION_STEPS = 4
FEEDBACK_BUFFER_SIZE = 5  # how many feedback corrections until we train

DEVICE_CONFIG_PATH = "/kaggle/input/cisco-automation/PE1-Cisco.txt"
COMPLIANCE_XLS_PATH = "/kaggle/input/cisco-automation/Cisco IOS XE Switch RTR Security Technical Implementation Guide-MAC-3_Sensitive.xls"

In [6]:
##############################################################################
# 2) LOAD MODELS
##############################################################################
print("Loading embedding model...")
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

print("Loading main LLM (deepseek-ai/deepseek-llm-7b-chat)...")
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-llm-7b-chat")
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/deepseek-llm-7b-chat",
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder="offload"
)
# Optional LoRA
try:
    lora_config = LoraConfig(
        r=LORA_RANK,
        lora_alpha=LORA_ALPHA,
        target_modules=TARGET_MODULES,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
except NameError:
    print("Skipping LoRA setup...")

tokenizer.pad_token = tokenizer.eos_token

Loading embedding model...
Loading main LLM (deepseek-ai/deepseek-llm-7b-chat)...


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.29s/it]


trainable params: 3,932,160 || all params: 6,914,297,856 || trainable%: 0.0569


In [7]:
##############################################################################
# 3) ONLINE FEEDBACK DATA
##############################################################################
try:
    with open("feedback_dataset.json", "r") as f:
        feedback_data = json.load(f)
except FileNotFoundError:
    feedback_data = {"examples": []}

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = TrainingArguments(
    output_dir="./lora_adapters",
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    max_grad_norm=0.3,
    save_strategy="epoch",
    logging_steps=10,
    remove_unused_columns=True
)

def online_fine_tune():
    """
    Fine-tunes the model with user feedback once we have enough samples.
    """
    if len(feedback_data["examples"]) < BATCH_SIZE:
        return

    formatted_data = []
    for example in feedback_data["examples"]:
        in_ids = tokenizer.encode(example["input"])
        out_ids = tokenizer.encode(example["output"])
        formatted_data.append({
            "input_ids": in_ids,
            "attention_mask": [1]*len(in_ids),
            "labels": out_ids
        })

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=formatted_data,
        data_collator=data_collator
    )
    trainer.train()
    
    model.save_pretrained("./lora_adapters")
    feedback_data["examples"] = []
    with open("feedback_dataset.json", "w") as f:
        json.dump(feedback_data, f)



E0000 00:00:1740487085.019284      10 common_lib.cc:621] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:239


In [8]:
##############################################################################
# 4) DOCUMENTS: RAG Knowledge Base
##############################################################################
# These are placeholders. Add your own docs or load them from files.
documents = [
    {
        "id": "example_doc",
        "text": """This is an example doc, with some generic info about Cisco or networking."""
    }
]

# Example troubleshooting steps from your PNG
troubleshooting_doc = {
    "id": "troubleshooting_steps",
    "text": """TROUBLESHOOTING STEPS:
1. Check physical connectivity (cables, ports).
2. Verify interface status with 'show interface' or 'show ip interface brief'.
3. Check routing table for correct routes.
4. Check firewall policy if packets are being dropped.
5. Validate application server is reachable (ping, traceroute).
"""
}
documents.append(troubleshooting_doc)

In [18]:
##############################################################################
# 5) FAISS INDEX
##############################################################################
print("Building FAISS index...")

texts = [doc["text"] for doc in documents]
doc_ids = [doc["id"] for doc in documents]

embeddings = embed_model.encode(texts, convert_to_numpy=True)
embedding_dim = embeddings.shape[1]

index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings)


Building FAISS index...


In [10]:
##############################################################################
# 6) COMPLIANCE CHECK LOGIC
##############################################################################
def load_compliance_rules(xls_path):
    """
    Reads compliance rules from an Excel spreadsheet.
    """
    df = pd.read_excel(xls_path, sheet_name=0)
    rules = []
    for idx, row in df.iterrows():
        rule_id = row.get("RuleID", f"Rule{idx+1}")
        desc = row.get("Description", "")
        pattern = row.get("RegexPattern", "")
        expected = row.get("ExpectedValue", "present")
        rules.append({
            "id": rule_id,
            "desc": desc,
            "regex": pattern,
            "expected": expected
        })
    return rules

def load_device_config(txt_path):
    """
    Reads device config from a text file.
    """
    with open(txt_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    return [line.strip() for line in lines]

def check_compliance(config_lines, rules):
    """
    Simple presence/absence check using regex.
    """
    report = []
    for rule in rules:
        rule_id = rule['id']
        desc = rule['desc']
        pattern = rule['regex']
        expected = rule['expected'].lower()

        found_match = any(re.search(pattern, line) for line in config_lines)

        if expected == "present":
            status = "PASS" if found_match else "FAIL"
        elif expected == "absent":
            status = "PASS" if not found_match else "FAIL"
        else:
            status = "UNKNOWN"

        report.append({
            "RuleID": rule_id,
            "Description": desc,
            "Status": status
        })
        
        return report
        
def format_compliance_report(report):
    lines = ["\n=== Compliance Report ==="]
    for item in report:
        lines.append(f"{item['RuleID']}: {item['Description']} => {item['Status']}")
    return "\n".join(lines)

def handle_compliance_request(device_config_path, xls_path):
    config_lines = load_device_config(device_config_path)
    rules = load_compliance_rules(xls_path)
    results = check_compliance(config_lines, rules)
    return format_compliance_report(results)

In [11]:
##############################################################################
# 7) RETRIEVAL & GENERATION
##############################################################################
def retrieve_faiss(query, model, index, doc_ids, texts, top_k=3):
    query_vec = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vec, top_k)
    results = []
    for rank, i in enumerate(indices[0]):
        results.append({
            "id": doc_ids[i],
            "text": texts[i],
            "distance": distances[0][rank]
        })
    return results

def chain_of_thought_prompt(conversation_text):
    """
    Build a chain-of-thought style prompt with the entire conversation included.
    """
    # We'll add a short system instruction encouraging step-by-step reasoning:
    system_instruction = (
        "You are a helpful assistant that uses chain-of-thought reasoning internally.\n"
        "Respond to the user with a concise final answer.\n\n"
    )

    # Then we add the conversation so far:
    # conversation_text is e.g. "User: blah\nAgent: blah\nUser: blah\nAgent: blah ..."

    # We'll end with "Chain-of-thought:" to get step-by-step reasoning
    prompt = f"{system_instruction}{conversation_text}\nChain-of-thought:"
    return prompt

def generate_llm_response(conversation_text):
    """
    Creates a chain-of-thought style prompt from the conversation,
    then returns only the final answer.
    """
    prompt = chain_of_thought_prompt(conversation_text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # The model might produce: "...Chain-of-thought: [some reasoning]\nFinal Answer: [some final]"
    final_answer = raw_output
    if "Final Answer:" in raw_output:
        final_answer = raw_output.split("Final Answer:", 1)[-1].strip()
    elif "Chain-of-thought:" in raw_output:
        # fallback if we never saw "Final Answer:"
        final_answer = raw_output.split("Chain-of-thought:", 1)[-1].strip()

    return final_answer

def parse_correctness(answer):
    answer_lower = answer.lower()
    positive_terms = ['correct', 'yes', 'proper']
    negative_terms = ['incorrect', 'no', 'wrong', 'mismatch']
    positive_count = sum(term in answer_lower for term in positive_terms)
    negative_count = sum(term in answer_lower for term in negative_terms)
    return 1 if positive_count > negative_count else 0


In [17]:
##############################################################################
# 8) MAIN CONVERSATION LOOP
##############################################################################
def run_conversational_rag():
    """
    Multi-turn conversation approach, like ChatGPT.
    We store conversation history in a list of (role, text).
    On each new user message:
      - check if user wants compliance or troubleshooting
      - if so, handle that, then append to conversation
      - else do retrieval + generation in the style of chain-of-thought
    The conversation ends only when user says 'exit' or 'quit'.
    """
    print("Starting RAG Chatbot in a multi-turn conversation style. Type 'exit' to quit.")
    # We'll keep track of the entire conversation as a list of (role, text).
    conversation = []

    # Helper function to get conversation as a text string:
    def conversation_to_text(convo):
        """
        Convert conversation list to a text block: "User: ...\nAgent: ...\nUser: ...\nAgent: ..."
        """
        lines = []
        for turn in convo:
            if turn["role"] == "user":
                lines.append(f"User: {turn['text']}")
            else:
                lines.append(f"Agent: {turn['text']}")
        return "\n".join(lines)

    while True:
        user_input = input("\nUser: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break

        # Append user turn to conversation
        conversation.append({"role": "user", "text": user_input})

        # Check if "compliance" is in user_input => compliance check
        if "compliance" in user_input.lower():
            compliance_text = handle_compliance_request(DEVICE_CONFIG_PATH, COMPLIANCE_XLS_PATH)
            # Add the compliance_text as the "agent" response
            conversation.append({"role": "agent", "text": compliance_text})
            print(f"\nAgent:\n{compliance_text}")
            continue

        # Check if "troubleshoot" or "issue" => troubleshooting
        if "troubleshoot" in user_input.lower() or "issue" in user_input.lower():
            # We'll do retrieval for user_input to gather troubleshooting doc:
            retrieved = retrieve_faiss(user_input, embed_model, index, doc_ids, texts, top_k=3)
            context = "\n\n".join(doc['text'] for doc in retrieved)
            # Now build the conversation text plus context
            conversation_text = conversation_to_text(conversation)
            # We'll just append the context as if the agent "knows" it
            conversation_text_with_context = f"{conversation_text}\nAdditional context:\n{context}"
            # Generate chain-of-thought style answer
            agent_answer = generate_llm_response(conversation_text_with_context)
            conversation.append({"role": "agent", "text": agent_answer})
            print(f"\nAgent:\n{agent_answer}")
            # Optionally ask for feedback
            feedback = input("Is this correct? (y/n or Enter to skip): ").lower()
            if feedback == "n":
                correction = input("Please provide the correct answer: ")
                # Store training data for online finetuning
                input_prompt_for_finetune = f"{conversation_text_with_context}\nChain-of-thought:"
                feedback_data["examples"].append({
                    "input": input_prompt_for_finetune,
                    "output": correction
                })
                if len(feedback_data["examples"]) >= FEEDBACK_BUFFER_SIZE:
                    online_fine_tune()
                agent_answer = correction
                conversation[-1]["text"] = agent_answer
                print(f"\nCorrected Agent Answer:\n{agent_answer}")
            continue

        # Otherwise, normal RAG
        retrieved = retrieve_faiss(user_input, embed_model, index, doc_ids, texts, top_k=3)
        context = "\n\n".join(doc['text'] for doc in retrieved)
        conversation_text = conversation_to_text(conversation)
        # Add the retrieved context
        conversation_text_with_context = f"{conversation_text}\nAdditional context:\n{context}"
        # Generate chain-of-thought style answer
        agent_answer = generate_llm_response(conversation_text_with_context)
        conversation.append({"role": "agent", "text": agent_answer})

        print(f"\nAgent:\n{agent_answer}")

        # Optionally prompt for feedback
        feedback = input("Was this answer correct? (y/n or Enter to skip): ").lower()
        if feedback == "n":
            correction = input("Please provide the correct answer: ")
            # Store training data
            input_prompt_for_finetune = f"{conversation_text_with_context}\nChain-of-thought:"
            feedback_data["examples"].append({
                "input": input_prompt_for_finetune,
                "output": correction
            })
            if len(feedback_data["examples"]) >= FEEDBACK_BUFFER_SIZE:
                online_fine_tune()
            agent_answer = correction
            conversation[-1]["text"] = agent_answer
            print(f"\nCorrected Agent Answer:\n{agent_answer}")

    # End of conversation loop
    # Save feedback data if any
    with open("feedback_dataset.json", "w") as f:
        json.dump(feedback_data, f)


In [13]:
##############################################################################
# 9) OPTIONAL EVALUATION
##############################################################################
def evaluate_agent(y_true, y_pred):
    if not y_true or not y_pred:
        print("No data available for evaluation.")
        return

    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_true, y_pred) if len(set(y_true)) > 1 else "N/A",
        "confusion_matrix": confusion_matrix(y_true, y_pred),
        "classification_report": classification_report(y_true, y_pred)
    }

    plt.figure(figsize=(10, 4))

    plt.subplot(1, 2, 1)
    sns.heatmap(metrics["confusion_matrix"], annot=True, fmt="d", cmap="Blues",
                xticklabels=['Incorrect', 'Correct'], yticklabels=['Incorrect', 'Correct'])
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")

    plt.subplot(1, 2, 2)
    metric_names = ['Accuracy', 'Precision', 'Recall', 'F1']
    metric_values = [metrics['accuracy'], metrics['precision'], metrics['recall'], metrics['f1']]
    sns.barplot(x=metric_names, y=metric_values, palette="viridis")
    plt.ylim(0, 1)
    plt.title("Performance Metrics")

    plt.tight_layout()
    plt.show()

    print("Classification Report:")
    print(metrics["classification_report"])
    print("\nROC AUC Score:", metrics["roc_auc"])

In [14]:
##############################################################################
# 10) MAIN
##############################################################################
if __name__ == "__main__":
    # Just run the conversation loop
    run_conversational_rag()

Starting RAG Chatbot in a multi-turn conversation style. Type 'exit' to quit.



User:  Hey, can you generate a compliance report for our PE1‐Cisco device against the latest STIG Excel?



Agent:

=== Compliance Report ===
Rule1:  => PASS



User:  That’s helpful. Could you specifically highlight any high‐severity rules that failed the check if there any?



Agent:
1. Ask if there are any high-severity rules that failed the check.
2. If yes, provide an explanation for each high-severity failure.
3. If no, provide a concise message that all rules have passed.

Agent:There are no high-severity rules that have failed the check. Your device is compliant with the latest STIG Excel for PE1-Cisco.


...


Agent:
- The user is experiencing issues with a router ping to 8.8.8.8.
- The first step is to check the physical connectivity of the cables and ports.
- Verify the interface status with 'show interface' or 'show ip interface brief'.
- Check the routing table for correct routes.
- Check the firewall policy if packets are being dropped.
- Validate the application server is reachable.

This doc should be helpful for troubleshooting issues with a router ping to 8.8.8.8.


Was this answer correct? (y/n or Enter to skip):  n
Please provide the correct answer:  Thank you. GoodBye!



Corrected Agent Answer:
Thank you. GoodBye!



User:  exit


Goodbye!
